### Importation des identifiants des villes

In [31]:
import pandas as pd

municipalities = pd.read_csv(
    'data/2-0-municipalities.csv',
    index_col=0
)

municipalities

,name
insee_code,
80001,Abbeville
47001,Agen
13001,Aix-en-Provence
2A004,Ajaccio
73011,Albertville
...,...
47323,Villeneuve-sur-Lot
14762,Vire Normandie
51649,Vitry-le-François


### Importation depuis [insee.fr](https://insee.fr)

Les pages du site qui donnent les statistiques commune par commune ont une URL standardisée de la forme:

`https://www.insee.fr/fr/statistiques/2011101?geo=COM-...`

... où `...` représente le code INSEE de la commune.

Par ailleurs, ces pages sont structurées de façon identique, et il est donc possible via la bibliothèque `beautifulsoup` d'y aller piocher les informations utiles.

Certains communes cependant, typiquement des communes de l'outre-mer, ont des pages avec des informations structuréee différemment. Avec une **gestion des erreurs** du type `try -> except`, nous testons donc si l'information est présente sur le site, et renvoyons une valeur manquante le cas échéant.

La fonction ci-dessous retourne les données pour **_une_** ville:

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen 

def read_insee(insee_code):
    
    url = "https://www.insee.fr/fr/statistiques/2011101?"+\
        "geo=COM-"+insee_code

    with urlopen(url) as page:

        parsed_page = BeautifulSoup(page)
        
        # Part des 60 à 74 ans et des 75 ans ou plus

        try:
            age_table_rows = parsed_page.find(id="produit-tableau-POP_T0").find_all('tr')
            rate_60_74       = age_table_rows[6].find_all('td')[5].text.replace(",", ".")
            rate_75_and_over = age_table_rows[7].find_all('td')[5].text.replace(",", ".")
            rate_60_and_over = float(rate_60_74 ) + float( rate_75_and_over )
            print(' ✅ rate', end="")
        except AttributeError :
            # catches cases where the table is missing
            print(' ❌ rate', end="")
            rate_60_and_over = pd.NA


        # Médiane du revenu disponible par unité de consommation (en euros)
        
        try:
            income_table_rows = parsed_page.find(id="produit-tableau-REV_T1").find_all('tr')
            median_income = income_table_rows[3].find('td').text.replace('\xa0', '')
            median_income = float( median_income )
            print(' ✅ inco', end="")
        except AttributeError :
            # catches cases where the table is missing
            print(' ❌ inco', end="")
            median_income = pd.NA

    return({
        "rate_60_and_over" : rate_60_and_over,
        "median_income" : median_income
    })
    

Démonstration sur la ville de Poitiers:

In [ ]:
poitiers = municipalities[ municipalities["name"]=="Poitiers" ]
insee_code_poitiers = poitiers.index[0]
read_insee(insee_code_poitiers)

 ✅ rate ✅ inco

{'rate_60_and_over': 20.1, 'median_income': 19000.0}

Ensuite il ne reste plus qu'à exécuter la fonction sur l'ensemble des communes ...

In [ ]:
# WARNING !!! TAKES ~1 HOUR TO RUN !!!

socio_economic = []
for insee_code, values in municipalities.iterrows() :
    print('📍 '+ values[0].ljust(32), end='')
    socio_economic.append( read_insee(insee_code) )
    print("")

📍 Abbeville                        ✅ rate ✅ inco
📍 Agen                             ✅ rate ✅ inco
📍 Aix-en-Provence                  ✅ rate ✅ inco
📍 Ajaccio                          ✅ rate ✅ inco
📍 Albertville                      ✅ rate ✅ inco
📍 Albi                             ✅ rate ✅ inco
📍 Alençon                          ✅ rate ✅ inco
📍 Alès                             ✅ rate ✅ inco
📍 Altkirch                         ✅ rate ✅ inco
📍 Ambert                           ✅ rate ✅ inco
📍 Amiens                           ✅ rate ✅ inco
📍 Ancenis-St-Géréon                ✅ rate ✅ inco
📍 Angers                           ✅ rate ✅ inco
📍 Angoulême                        ✅ rate ✅ inco
📍 Annecy                           ✅ rate ✅ inco
📍 Apt                              ✅ rate ✅ inco
📍 Arcachon                         ✅ rate ✅ inco
📍 Argelès-Gazost                   ✅ rate ✅ inco
📍 Argentan                         ✅ rate ✅ inco
📍 Argenteuil                       ✅ rate ✅ inco
📍 Arles             

... puis à convertir le résultat en tableau de données :

In [ ]:
socio_economic_df = pd.DataFrame(socio_economic)
socio_economic_df.set_index(municipalities.index, inplace=True)
socio_economic_df.to_csv("data/2-2-socio-economic-from-insee.csv")

### Vérification

In [25]:
import pandas as pd
socio_economic_df = pd.read_csv("data/2-2-socio-economic-from-insee.csv", index_col=0)
socio_economic_df


,rate_60_and_over,median_income
insee_code,,
80001,29.8,17910.0
47001,24.1,18180.0
13001,24.4,24340.0
2A004,28.6,21480.0
73011,28.3,19740.0
...,...,...
47323,36.7,18420.0
14762,34.7,20430.0
51649,30.6,16650.0


In [30]:
_deepnote_run_altair(socio_economic_df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"rate_60_and_over","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"median_income","type":"quantitative","sort":null,"scale":{"type":"sqrt","zero":false}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f812232-8ed1-4e89-8f48-597e10058637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>